In [3]:
from TSPLIPLoader import TPSLIBLoader

# berlin52 = TPSLIBLoader('berlin52')
burma14 = TPSLIBLoader('burma14')


burma14.nodes

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [4]:
burma14.get_solution

3323

In [5]:
from HSFFPKO import HSFFPKO, universal_tsp_wrapper
from EHO import elephant_herd_optimization
from ACO import ant_colony
from SA import simulated_annealing
from SAI import simulated_annealing_improved
from GWO import GreyWolfTSP
from PSO import hybrid_pso_tsp
from GA import genetic_alg 

num_nodes = len(burma14.nodes)
graph = burma14.to_graph()
matrix = burma14.to_matrix()

Shape: (10, 10)
Sample: [ 0. 20. 13. 21. 16.]...


In [ ]:
import time 
import numpy as np

data = {
    'HSFFPKO': [],
    'EHO': [],
    'ACO': [],
    'SA' : [],
    'SAI': [],
    'GWO': [],
    'PSO': [],
    'GA': []
    
}

N = 10
for _ in range(N):
    start = time.perf_counter()
    output = HSFFPKO(
        Popsize=50,
        Maxiteration=500,
        LB=np.zeros(num_nodes),
        UB=np.ones(num_nodes) * 20,
        Dim=num_nodes,
        Fobj=universal_tsp_wrapper(matrix),  # MATRIX
        scouts_rate=0.15,
        num_flocks=8
    )[0] 
    end = time.perf_counter()
    data['HSFFPKO'].append([output,end - start])
    
    start = time.perf_counter()
    output = elephant_herd_optimization(graph)[1]
    end = time.perf_counter()
    data['EHO'].append([output,end - start])

    start = time.perf_counter()
    output = ant_colony(graph)[1]
    end = time.perf_counter()
    data['ACO'].append([output,end - start])

    start = time.perf_counter()
    output = simulated_annealing(graph, temperature=1000, alpha=0.995, iterations=1000)[1]
    end = time.perf_counter()
    data['SA'].append([output,end - start])

    start = time.perf_counter()
    output = simulated_annealing_improved(graph, iter_per_temp=100)[0]
    end = time.perf_counter()
    data['SAI'].append([output,end - start])

    gwo = GreyWolfTSP(graph)
    start = time.perf_counter()
    output = gwo.optimize()[1]
    end = time.perf_counter()
    data['GWO'].append([output,end - start])

    start = time.perf_counter()
    output = hybrid_pso_tsp(graph)[1]
    end = time.perf_counter()
    data['PSO'].append([output,end - start])

    start = time.perf_counter()
    output = genetic_alg(graph, pop_size=100, mutation_rate=0.01, generations=50)[1]
    end = time.perf_counter()
    data['GA'].append([output,end - start])




In [ ]:
import pandas as pd

best_cost = burma14.get_solution

cost_dict = {k: [x[0] for x in v] for k, v in data.items()}
df_cost = pd.DataFrame(cost_dict)

runtime_dict = {k: [x[1] for x in v] for k, v in data.items()}
df_runtime = pd.DataFrame(runtime_dict)

# Get the relative error
def relative_error(costs, best_cost):
    return (costs - best_cost) / best_cost

rel_error_dict = {k: relative_error(np.array(v), best_cost) for k, v in cost_dict.items()}
df_rel_error = pd.DataFrame(rel_error_dict)

cols = ['HSFFPKO', 'EHO', 'ACO', 'SA', 'SAI', 'GWO', 'PSO', 'GA']

df = pd.DataFrame(
    np.vstack([df_cost.mean(), df_runtime.mean(), df_cost.min(), df_rel_error.mean() * 100, df_rel_error.min() * 100]),
    columns=cols,
    index=['Mean Cost', 'Mean Runtime', 'Min Cost', 'Mean Rel Error', 'Min Rel Error']
).transpose()

df.sort_values(by='Mean Cost', inplace=True)

df

,Mean Cost,Mean Runtime,Min Cost,Mean Rel Error,Min Rel Error
HSFFPKO,3373.0,1.269904,3323.0,1.504664,0.0
